# OpenBnB MCP सर्व्हर इंटिग्रेशनसह Semantic Kernel

हे नोटबुक वास्तविक OpenBnB MCP सर्व्हरसह Semantic Kernel कसा वापरायचा हे दाखवते जेणेकरून MCPStdioPlugin वापरून वास्तविक Airbnb निवासस्थान शोधता येतील. LLM ऍक्सेससाठी, ते Microsoft Foundry वापरते. आपले पर्यावरण बदल सेट करण्यासाठी, आपण [Setup Lesson](/00-course-setup/README.md) अनुसरू शकता.


## आवश्यक पॅकेजेस आयात करा


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP प्लगइन कनेक्शन तयार करणे

आम्ही MCPStdioPlugin वापरून [OpenBnB MCP सर्व्हर](https://github.com/openbnb-org/mcp-server-airbnb) शी कनेक्ट करू. हा सर्व्हर @openbnb/mcp-server-airbnb पॅकेजद्वारे Airbnb शोध कार्यक्षमता प्रदान करतो.


## क्लायंट तयार करणे

या नमुन्यात, आपण आपल्या LLM ऍक्सेससाठी Microsoft Foundry वापरणार आहोत. आपल्या पर्यावरण चल योग्यरित्या सेट केल्याची खात्री करा.


## Environment Configuration

Azure OpenAI सेटिंग्ज कॉन्फिगर करा. खालील पर्यावरण चल सेट असल्याची खात्री करा:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP एकत्रीकरण समजून घेणे

हा नोटबुक **खऱ्या OpenBnB MCP सर्व्हरशी** कनेक्ट होतो जो प्रत्यक्ष Airbnb शोध कार्यक्षमता प्रदान करतो.

### हे कसे कार्य करते:

1. **MCPStdioPlugin**: MCP सर्व्हरशी मानक इनपुट/आउटपुट संवाद वापरतो
2. **खरा NPM पॅकेज**: npx द्वारे `@openbnb/mcp-server-airbnb` डाउनलोड आणि चालवतो
3. **थेट डेटा**: त्यांच्या API कडून प्रत्यक्ष Airbnb मालमत्ता डेटा परत करतो
4. **फंक्शन शोध**: एजंट स्वयंचलितपणे MCP सर्व्हरकडून उपलब्ध फंक्शन्स शोधतो

### उपलब्ध फंक्शन्स:

OpenBnB MCP सर्व्हर सामान्यतः प्रदान करतो:
- **search_listings** - स्थान आणि निकषांनुसार Airbnb मालमत्ता शोधा
- **get_listing_details** - विशिष्ट मालमत्तांचे सविस्तर माहिती मिळवा
- **check_availability** - विशिष्ट तारखांसाठी उपलब्धता तपासा
- **get_reviews** - मालमत्तांसाठी पुनरावलोकने मिळवा
- **get_host_info** - मालमत्ता तळकांच्या माहिती मिळवा

### पूर्वअटी:

- आपल्या सिस्टमवर **Node.js** स्थापीत असणे आवश्यक आहे
- MCP सर्व्हर पॅकेज डाउनलोड करण्यासाठी **इंटरनेट कनेक्शन**
- **NPX** उपलब्ध असणे (Node.js सह येते)

### कनेक्शन तपासणी:

आपण खालीलप्रमाणे MCP सर्व्हरमध्ये मॅन्युअली कनेक्ट करून तपासणी करू शकता:
```bash
npx -y @openbnb/mcp-server-airbnb
```

यामुळे OpenBnB MCP सर्व्हर डाउनलोड आणि चालू होईल, ज्याला Semantic Kernel नंतर प्रत्यक्ष Airbnb डेटासाठी कनेक्ट होतो.


## OpenBnB MCP सर्व्हरसह एजंट चालविणे

आता आपण AI एजंट चालवणार आहोत जो OpenBnB MCP सर्व्हरशी कनेक्ट होतो आणि स्टॉकहोममधील 2 प्रौढ आणि 1 मुलासाठी खऱ्या Airbnb निवासस्थानांची शोध घेतो. शोध निकष बदलण्यासाठी `user_inputs` यादीमध्ये बदल करणे मोकळे आहे.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# सारांश
अभिनंदन! आपण यशस्वीरित्या एक AI एजंट तयार केला आहे जो Model Context Protocol (MCP) वापरून वास्तविक जगातील निवास शोधाशी एकत्रित होतो:

## वापरलेली तंत्रज्ञान:
- Semantic Kernel - Azure OpenAI सह बुद्धिमान एजंट तयार करण्यासाठी
- Microsoft Foundry - LLM क्षमता आणि चॅट पूर्णतेसाठी
- MCP (Model Context Protocol) - प्रमाणित साधन समाकलनासाठी
- OpenBnB MCP Server - वास्तविक Airbnb शोध कार्यक्षमता साठी
- Node.js/NPX - बाह्य MCP सर्व्हर चालवण्यासाठी

## आपण जे शिकलात:
- MCP समाकलन: Semantic Kernel एजंट्सना बाह्य MCP सर्व्हरशी जोडणे
- वास्तविक-वेळ डेटा प्रवेश: लाइव API द्वारे प्रत्यक्ष Airbnb मालमत्ता शोधणे
- प्रोटोकॉल संवाद: एजंट आणि MCP सर्व्हर यांच्यामध्ये stdio संवाद वापरणे
- फंक्शन शोध: MCP सर्व्हरकडून उपलब्ध फंक्शन्स आपोआप शोधणे
- स्ट्रिमिंग प्रतिसाद: फंक्शन कॉल्स रिअल-टाइममध्ये कैप्चर करणे आणि लॉग करणे
- HTML रेंडरिंग: स्टाइल केलेल्या टेबल्स आणि इंटरऐक्टिव्ह डिस्प्लेसह एजंट प्रतिसादांचे स्वरूपण करणे

## पुढील पावले:
- अतिरिक्त MCP सर्व्हर्स समाकलित करणे (हवामान, फ्लाइट्स, रेस्टॉरंट्स)
- MCP आणि A2A प्रोटोकॉल एकत्र करून मल्टी-एजंट सिस्टम तयार करणे
- आपल्या स्वतःच्या डेटास्रोतांसाठी सानुकूल MCP सर्व्हर्स तयार करणे
- सत्रांदरम्यान सतत संभाषण स्मृती अमलात आणणे
- MCP सर्व्हर ऑर्केस्ट्रेशनसह Azure Functions मध्ये एजंटची तैनाती करणे
- वापरकर्त्यांचे प्रमाणीकरण आणि बुकिंग क्षमता जोडणे


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**स्पष्टीकरण**:
हा दस्तऐवज AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) चा वापर करून अनुवादित केला आहे. आम्ही अचूकतेसाठी प्रयत्न करतो, तरी कृपया लक्षात घ्या की स्वयंचलित अनुवादांमध्ये चुका किंवा अचूकतेच्या त्रुटी असू शकतात. मूळ दस्तऐवज त्याच्या स्थानिक भाषेत अधिकृत स्रोत मानला पाहिजे. महत्त्वपूर्ण माहितीसाठी व्यावसायिक मानव अनुवाद शिफारस केला जातो. या अनुवादाच्या वापरामुळे उद्भवलेल्या कोणत्याही गैरसमजुती किंवा चुकीच्या अर्थलागी आम्ही जबाबदार नाही.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
